In [87]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import cPickle as pickle
import csv
from collections import Counter
import rdkit.Chem.AllChem

In [2]:
column_names = pickle.load(open('column_names.p', 'rb'))

In [ ]:
%%time
df_train = pd.read_csv('array_touse_train.csv', names=column_names)

In [38]:
%%time
with open('val_train_split.csv', 'rb') as f:
    reader = csv.reader(f)
    valid_mask = map(lambda lst : int(float(lst[0])) == 0, list(reader))
print len(valid_mask)
print sum(valid_mask)

Wall time: 11.5 s


In [ ]:
df_valid = df_train.loc[valid_mask]
df_train = df_train.loc[~valid_mask]

In [51]:
# LOAD GAP HERE SOMEHOW, INTO AN ARRAY CALLED Y_TRAIN AND AN ARRAY CALLED Y_VALID

In [ ]:
X_train = df_train.values
X_valid = df_valid.values

In [ ]:
%%time
LR = LinearRegression()
LR.fit(X_train, Y_train)
rmse_linear = mean_squared_error(Y_valid, LR.predict(X_valid))
print rmse_linear

In [ ]:
%%time
alphas = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
RR = RidgeCV(alphas=alphas, store_cv_values=True)
RR.fit(X_train, Y_train)
best_alpha = RR.alpha_
print 'Optimal alpha:', best_alpha
plt.plot(np.log(alphas), RR.cv_values_.mean(axis=0))
plt.xlabel('log(alpha)')
plt.ylabel('mean squared error')

RR = RidgeCV(alphas = [best_alpha*0.3, best_alpha, best_alpha*3])
RR.fit(X_train, Y_train)
best_alpha = RR.alpha_
print 'New optimal alpha:', best_alpha

rmse_ridge = mean_squared_error(Y_valid, RR.predict(X_valid))
print rmse_ridge

In [ ]:
%%time
alphas = [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]
LassoModel = LassoCV(alphas = alphas)
LassoModel.fit(X_train, Y_train)
best_alpha = LassoModel.alpha_
print 'Optimal alpha:', best_alpha
plt.plot(np.log(alphas), LassoModel.mse_path_.mean(axis=-1))
plt.xlabel('log(alpha)')
plt.ylabel('mean squared error')

LassoModel = LassoCV(alphas = [bset_alpha*0.3, best_alpha, best_alpha*3])
LassoModel.fit(X_train, Y_train)
best_alpha = LassoModel.alpha_
print 'New optimal alpha:', best_alpha

rmse_lasso = mean_squared_error(Y_valid, LassoModel.predict(X_valid))
print rmse_lasso

In [ ]:
%%time
alphas = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
l1_ratios = [0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.99]
EN = ElasticNetCV(l1_ratio=l1_ratios, alphas=alphas)
EN.fit(X_train, Y_train)
print 'Optimal alpha:', EN.alpha_
print 'Optimal L1 ratio:', EN.l1_ratio_
print mean_squared_error(Y_valid, EN.predict(X_valid))